In [18]:
import pandas as pd

In [19]:
data = pd.read_csv('data\Airlines.csv')
data

,id,Airline,Flight,AirportFrom,AirportTo,DayOfWeek,Time,Length,Delay
0,1,CO,269,SFO,IAH,3,15,205,1
1,2,US,1558,PHX,CLT,3,15,222,1
2,3,AA,2400,LAX,DFW,3,20,165,1
3,4,AA,2466,SFO,DFW,3,20,195,1
4,5,AS,108,ANC,SEA,3,30,202,0
...,...,...,...,...,...,...,...,...,...
539378,539379,CO,178,OGG,SNA,5,1439,326,0
539379,539380,FL,398,SEA,ATL,5,1439,305,0
539380,539381,FL,609,SFO,MKE,5,1439,255,0
539381,539382,UA,78,HNL,SFO,5,1439,313,1


In [20]:
df = data.copy()
df.drop(['id'], axis=1, inplace=True)
df.columns = df.columns.str.lower()

### Get a complemetar data about the Airport codes

In [21]:
df_airpot_code = pd.read_csv('data\GlobalAirportDatabase.txt', delimiter=':', header=None )
df_airpot_code.columns = ['ICAO_code','IATA_code','airport_name','city_town','country','latitude_degrees',
                          'Latitude_Minutes','Latitude_Seconds', 'Latitude Direction', 'Longitude Degrees', 
                          'Longitude Minutes', 'Longitude Seconds', 'Longitude Direction', 'Altitude', 
                          'Latitude Decimal Degrees', 'Longitude Decimal Degrees']

df_airpot_code = df_airpot_code[['IATA_code','country']].dropna()
df_airpot_code

,IATA_code,country
0,GKA,PAPUA NEW GUINEA
1,LAE,PAPUA NEW GUINEA
2,MAG,PAPUA NEW GUINEA
3,HGU,PAPUA NEW GUINEA
4,LAE,PAPUA NEW GUINEA
...,...,...
9276,HTN,CHINA
9278,URC,CHINA
9286,HRB,CHINA
9293,CGQ,CHINA


In [22]:
3511-3562 

-51

In [23]:
print({'code':len(df_airpot_code.IATA_code.unique()), 'country' : len(df_airpot_code.country.unique())},
f"\n num of IATA duplicated : {len(df_airpot_code[df_airpot_code.IATA_code.duplicated()].sort_values(by=['IATA_code']))}")

{'code': 3511, 'country': 220} 
 num of IATA duplicated : 51


In [24]:
# Remove duplicates and display the DataFrame without duplicates
df_airpot_code_without_duplicates = df_airpot_code.drop_duplicates(subset='IATA_code')

## Merge the IATA codes on df

In [25]:
# 539383 rows on df
merged_df = pd.merge(df, df_airpot_code_without_duplicates, left_on='airportfrom',right_on='IATA_code', how='left')\
    .rename(columns={'country': 'country_from'})
merged_df = merged_df.drop('IATA_code', axis=1)
merged_df = pd.merge(merged_df, df_airpot_code_without_duplicates, left_on='airportto',right_on='IATA_code', how='left')\
    .rename(columns={'country': 'country_to'})
merged_df = merged_df.drop('IATA_code', axis=1)
merged_df

,airline,flight,airportfrom,airportto,dayofweek,time,length,delay,country_from,country_to
0,CO,269,SFO,IAH,3,15,205,1,USA,USA
1,US,1558,PHX,CLT,3,15,222,1,USA,USA
2,AA,2400,LAX,DFW,3,20,165,1,USA,USA
3,AA,2466,SFO,DFW,3,20,195,1,USA,USA
4,AS,108,ANC,SEA,3,30,202,0,USA,USA
...,...,...,...,...,...,...,...,...,...,...
539378,CO,178,OGG,SNA,5,1439,326,0,USA,USA
539379,FL,398,SEA,ATL,5,1439,305,0,USA,USA
539380,FL,609,SFO,MKE,5,1439,255,0,USA,USA
539381,UA,78,HNL,SFO,5,1439,313,1,USA,USA


## Create a new feature

* international fly feature

In [26]:
df = merged_df.copy()
df['international'] = (df.country_from == df.country_to).astype('int')

merged_df = df.copy()
merged_df

,airline,flight,airportfrom,airportto,dayofweek,time,length,delay,country_from,country_to,international
0,CO,269,SFO,IAH,3,15,205,1,USA,USA,1
1,US,1558,PHX,CLT,3,15,222,1,USA,USA,1
2,AA,2400,LAX,DFW,3,20,165,1,USA,USA,1
3,AA,2466,SFO,DFW,3,20,195,1,USA,USA,1
4,AS,108,ANC,SEA,3,30,202,0,USA,USA,1
...,...,...,...,...,...,...,...,...,...,...,...
539378,CO,178,OGG,SNA,5,1439,326,0,USA,USA,1
539379,FL,398,SEA,ATL,5,1439,305,0,USA,USA,1
539380,FL,609,SFO,MKE,5,1439,255,0,USA,USA,1
539381,UA,78,HNL,SFO,5,1439,313,1,USA,USA,1


## Seach for the IATA codes outside the IATA_databse
* celaning nan values from Countries coulumns

I could not find other IATA values source, in orter to continue the work I droped the Nan lines on the merged_df

In [27]:
df_nan_country = merged_df[merged_df.isna().any(axis=1)]
nan_from = set(df_nan_country.loc[df_nan_country['country_from'].isna(), 'airportfrom'].unique())
nan_to = set(df_nan_country.loc[df_nan_country['country_to'].isna(), 'airportto'].unique())
missing_IATA = list(nan_from.union(nan_to))

In [28]:
merged_df_final = merged_df.dropna()
len(merged_df_final), len(df_nan_country), len(merged_df)

(478914, 60469, 539383)

In [29]:
539383-60469

478914

## Data transformation

### Prepare the df for modeling


In [30]:
df = merged_df_final.copy()
df = df.drop(['flight','airline', 'airportfrom', 'airportto',  'country_from', 'country_to'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 478914 entries, 0 to 539382
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype
---  ------         --------------   -----
 0   dayofweek      478914 non-null  int64
 1   time           478914 non-null  int64
 2   length         478914 non-null  int64
 3   delay          478914 non-null  int64
 4   international  478914 non-null  int32
dtypes: int32(1), int64(4)
memory usage: 20.1 MB


In [31]:
def convert_days_of_week(df):
    day_mapping = {1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6}
    df['dayofweek'] = df['dayofweek'].map(day_mapping)

    return df

df = convert_days_of_week(df)


In [32]:
#df['flight'] = df['flight'].astype('str')
df['dayofweek'] = df['dayofweek'].astype('str')
df['international'] = df['international'].astype('int8')
df['delay'] = df['delay'].astype('int8') 

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 478914 entries, 0 to 539382
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   dayofweek      478914 non-null  object
 1   time           478914 non-null  int64 
 2   length         478914 non-null  int64 
 3   delay          478914 non-null  int8  
 4   international  478914 non-null  int8  
dtypes: int64(2), int8(2), object(1)
memory usage: 15.5+ MB


# Save a model Dataset

In [33]:
df.to_parquet('data\data_to_model\data_model.parquet', engine='fastparquet', compression='snappy')